# Julia As A Glue Language

Welcome to this hands-on tutorial of using Julia as a glue language. For enabling julia in colab, we first need to jump start it from python.

In [ ]:
# Installation cell
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the the cell directly above this text), go to Colab's Edit menu and select Notebook settings from the drop down. Select *Julia 1.3* as the runtime and *GPU* as the hadware accelerator. If *Julia 1.3* is already selected, switch back and forth to another runtime type and back.

<br/>You should see somthing like this:

> ![alt text](https://drive.google.com/uc?id=1AeglaLmWI-zRXPCErofIZ4BH9zvPCwNy)
<br/>Click on SAVE
<br/>**We are ready to get going**





Now let's start to setup Julia environment by installing some Julia packages.

In [ ]:
#Julia 1.3 Environment
using Pkg
# julia foreign function interfaces
pkg"add PyCall; precompile;"
pkg"add RCall; precompile;"
pkg"add PyPlot; precompile;"
# other
pkg"add BenchmarkTools; precompile;"

# Python FFI - PyCall.jl

We start this hands-on with interfacing Python. For this we load the `PyCall` julia package like follows

In [ ]:
using PyCall
using BenchmarkTools

One awesome feature is that you can pass many julia objects directly to python. For example here a julia function into python optimize method.

In [ ]:
so = pyimport("scipy.optimize")
so.newton(x -> cos(x) - x, 1)

There is actually not much overhead involved, as we can investigate by benchmarking. In Julia we can easily benchmark with help of the macro ``BenchmarkTools.@benchmark``. In python the standard is way more inconvenient - you need to use ``timeit.timeit``.

In [ ]:
py"""timeit("time.sleep(1)", number=3, setup="import time;")"""  # timeit reports the total runtime

Note that multiline py syntax will always return ``nothing``, you need to use single line syntax with a single command to grab a python variable.

In [ ]:
py"""
from timeit import timeit
timeresult = timeit("so.newton(lambda x: cos(x) - x, 1)", number=10000, setup="from scipy import optimize as so; from math import cos;")
"""
py"timeresult" / 10000 * 1e6 # microsecond

In [ ]:
using BenchmarkTools
@benchmark so.newton(x -> cos(x) - x, 1)

You see, it is just a factor of 1.4 slower

In [ ]:
269.514 / 190.270

You can also interpolate julia expressions into python code

In [ ]:
# we need to load packages only once
py"""
from scipy import optimize as so
from math import cos
"""
# plain julia function with python internals
mynewton(f) = py"so.newton($f, 1)"

In [ ]:
mynewton(x -> cos(x) - x)

Numpy is completely supported. Julia arrays are passed directly without copying to python. Numpy arrays are copied by default, however this can be changed, for details see https://github.com/JuliaPy/PyCall.jl#arrays-and-pyarray

In [ ]:
py"""
import numpy as np
"""
py"np.sin(np.pi * $([1,4,6]))"

## Matplotlib 
Matplotlib is integrated to IJulia Kernel, thanks to the ``PyPlot.jl`` wrapper arround matplotlib and PyCall.jl

In [ ]:
using PyPlot
x = range(0; stop=2*pi, length=1000); y = sin.(3 * x + 4 * cos.(2 * x));
plot(x, y, color="red", linewidth=2.0, linestyle="--")  # plot function comes from PyPlot
title("A sinusoidally modulated sinusoid")  # same here

PyPlot.jl really only adds some julia wrappers as well as IJulia JupyterNotebook support for PyCall

In [ ]:
plt = pyimport("matplotlib.pyplot")
plt.plot(x, y, color="red", linewidth=2.0, linestyle="--")
plt.title("The same sinusoidally modulated sinusoid")

In [ ]:
py"""
import matplotlib.pyplot as plt
plt.plot($x, $y, color="red", linewidth=2.0, linestyle="--")
plt.title("And another title")
"""

Finally note that if you want to include python code in your julia module, you need to put it into the ``__init__`` function which executes extra module-initializations the first time the module is actually loaded. Otherwise you may encounter problems with Julia's precompilation.

For more information see https://github.com/JuliaPy/PyCall.jl

# Python FFI - it is your turn!

Task 1: Try out to pass julia arrays to numpy functions and get results back.

A good tutorial is the numpy quickstart in general, especially the linear algebra part
https://docs.scipy.org/doc/numpy/user/quickstart.html#linear-algebra

The histogram is also nice https://docs.scipy.org/doc/numpy/user/quickstart.html#histograms

Try to use julia arrays where possible

Task2: Try out parts of the scipy tutorial about image processing https://docs.scipy.org/doc/scipy/reference/tutorial/signal.html.

Try to use julia arrays where possible

# Shell Intermezzo
Julia has also good support for shell and filesystem.

The first thing you already saw is that in every julia shell you can press `;` to enter shell-mode where you can run shell commands directly.

In [ ]:
; pwd

Many standard shell commands have also julia equivalents

In [ ]:
@show pwd()
@show length(readdir("/"))

If you look for traversing a directory, use `walkdir`

In [ ]:
for (root, dirs, files) in walkdir(".")
    println("Directories in $root")
    for dir in dirs
        println(joinpath(root, dir)) # path to directories
    end
    println("Files in $root")
    for file in files
        println(joinpath(root, file)) # path to files
    end
end

For more on filesystem see https://docs.julialang.org/en/v1/base/file/ 

### Julia's support for subprocesses

For executing subprocesses you use backticks `` myprogram = `echo "hi"` `` and then `run(myprogram)` or `read(mypogram, String)`, or ``open``, ...

Pipe ``a |b |c`` is `pipeline(a, b, c)` and concurrent exuction `&` is `&`.

You can find the full documentation at https://docs.julialang.org/en/v1/manual/running-external-programs/

In [ ]:
# if you use `run` interactively, appending `;` lets you suppress the output of the process object
run(`ls /`);

In [ ]:
read(pipeline(`echo world` & `echo hello`, `sort`), String)

You can also rewire stdout (and stderr) to write to a file or other process instead

In [ ]:
run(pipeline(`ls`, "tmp.txt"))  # or long run(pipeline(`ls`, stdout = "tmp.txt))
run(`cat tmp.txt`);

To write to a process using `print` and the like, you need to `open` the command as an io process

In [ ]:
open(pipeline(`cat`, "file.txt"), "w") do io
  println(io, "line 3")
  write(io, "line 1\nline 2\n")
  write(io, "line 4
line 5
")
  write(io, """
  line 6
  line 7
  """)
  println(io, "line last")
end
run(`cat file.txt`);

# Shell Intermezzo - it's your turn

Task1 for you: order the above output lines 1 to "last" by using the shell's ``sort`` command

Task2. Follow this little awk tutorial https://www.linuxtechi.com/awk-command-tutorial-with-examples/
Importantly, their rendering is a bit off. I.e. if you see a line like this
```
linuxtechi@mail:~$ awk ‘{print;}’ awk_file
```
Which would give you error messages like
```
awk: 1: unexpected character 0xe2
awk: 1: unexpected character 0xe2
```
what you really want to execute is
```
awk '{print;}' awk_file
```
I.e. use true single quotes instead of the special unicode characters!!

In [ ]:
# first part is to somehow get this input data into a file called "awk_file"
s = """
Name,Marks,Max Marks
Ram,200,1000
Shyam,500,1000
Ghyansham,1000
Abharam,800,1000
Hari,600,1000
Ram,400,1000
"""

In [ ]:
# the run some of the awk commands

# R FFI - RCall.jl
The same unbelievable interaction goes for R.

In [ ]:
using RCall

In [ ]:
x = randn(10)
R"t.test($x)"

In [ ]:
a = 1:4
b = 1:3
rcopy(R"$a + $b")

In [ ]:
R"optim(0, $(x -> x-cos(x)), method='BFGS')"

In [ ]:
R"library(datasets); data(iris); summary(iris);"

In [ ]:
rcopy(R"iris")[1:10, :]

In [ ]:
R"plot(iris)"

For installing R libraries you need to use Julia's subprocesses. For instance, let's install the famous `dplyr` package.

In [ ]:
run(`R -e "install.packages('dplyr')"`)  # R"""install.packages(...)""" is not working https://github.com/JuliaInterop/RCall.jl/issues/341

# R FFI - it's your turn!

Try to follow this R exercise using the default Iris dataset https://rpubs.com/moeransm/intro-iris

Important Note: R references like `iris$sepal.width` don't work, because apparently RCall is case sensitive and the real name is `iris.Sepal.Width` as you can see in the `plot(iris)`.

In [ ]:
R"""
library(datasets)
data(iris)
head(iris)
"""

In [ ]:
# go on here

Dplyr comes also with a nice default dataset about starwars. Inspect it a little bit! E.g. by following https://dplyr.tidyverse.org/#usage

In [ ]:
rcopy(R"""starwars""")

In [ ]:
# TODO

# Cpp FFI - Cxx.jl

We are going to interface opencv via its cpp implementation. Currently there is no maintained julia wrapper for opencv (see e.g. [this discussion](https://discourse.julialang.org/t/how-to-properly-install-the-opencv-package-in-julia/24619)) however as you will see, using the Cpp interface directly is actually impressively simple.

First we need to install the Cpp interface, as well as some julia image library to show results.

In [ ]:
using Pkg
pkg"add Cxx; precompile;"
# julia images
pkg"add Images; precompile;"
pkg"add ImageIO; precompile;"
pkg"add ImageMagick; precompile;"
pkg"add ImageDraw; precompile;"

In [ ]:
using Cxx
using Libdl

The string-macro `cxx"..."` and the macro `@cxx`

In [ ]:
# specify imports
cxx""" #include<iostream> """  

# Declare a cpp function
cxx"""  
  void mycppfunction(int x) {   
    int y = 5;
    int z = x*y + 2;
    std::cout << "The number is " << z << std::endl;
  }
"""
# Convert C++ to Julia function
julia_function(x) = @cxx mycppfunction(x)

julia_function(10)

Declaring cpp functions has the disadvantage that you cannot simply rerun the cell above. It will throw an error that a respective cpp function is already defined. Try it out!

The solution is to use inline cpp code like we did with Python and R. However unlike R and python, which are interpreted by default, here we have to use a second cpp string-macro `icxx"..."`.

Remember like
- `icxx"..."` = interpreted cxx
- `cxx"..."` = standard cxx definitions

In [ ]:
function julia_function2(x)
  icxx"""
    int y = 5;
    int z = $x*y + 2;  // again we can simply interpolate julia values
    std::cout << "The number is " << z << std::endl;
  """
end

julia_function2(10);

Dealing with String

In [ ]:
function cppstring(str)
  chararray = pointer(str)
  icxx"""
    std::string s = $chararray;
    s;
  """
end

Importantly, standard interactive printing does not work for most of cxx.

In [ ]:
cppstring("hi"); # try it out by removing the semicolon

But nevertheless we can correctly pass cpp values again to cpp.

In [ ]:
icxx""" std::cout << $(cppstring("hello world")) << std::endl; """;

### OpenCV

I was in needed of using opencv, because I wanted to have a haarcascade-face-recognizer and in julia I couldn't find any implementation. OpenCV has one, and we will enable it in this section.

-------------

The next cell adds all opencv cpp libraries and headers to Julia using the standard package `Libdl`.

After this we can use opencv directly.

*Note that if you want to include this into your own julia package, make sure you run it in the ``__init__`` method so that it is executed only at runtime, as soon as you load your module.*

In [ ]:
ENV["PKG_CONFIG_ALLOW_SYSTEM_LIBS"] = true  # this is needed, as the lib library is excluded by default otherwise
pkg_config_libdir = strip(read(`pkg-config --libs-only-L opencv`, String))
cvlibdir = match(r"^-L([^ ]*)$", pkg_config_libdir)[1]

pkg_config_libs = strip(read(`pkg-config --libs-only-l opencv`, String))
extract_match(x) = match(r"^-l([^ ]*)$", x)[1]
cvlibs = extract_match.(split(pkg_config_libs))
for cvlib in cvlibs
    Libdl.dlopen(joinpath(cvlibdir,"lib$cvlib.so"), Libdl.RTLD_GLOBAL)
end

pkg_config_cflags = strip(read(`pkg-config --cflags opencv`, String))
cvheaderdir = dirname(match(r"^-I([^ ]*)$", pkg_config_cflags)[1])  # path which should end on "/include"

addHeaderDir(cvheaderdir, kind = C_System)
addHeaderDir(joinpath(cvheaderdir,"opencv"), kind = C_System )
addHeaderDir(joinpath(cvheaderdir, "opencv2"), kind = C_System )
addHeaderDir(joinpath(cvheaderdir, "opencv2", "core"), kind = C_System )

In [ ]:
# general opencv setup
cxx"""
#include <stdio.h>
#include <iostream>
#include <opencv2/core.hpp>
#include <opencv2/core/core.hpp>
#include "opencv2/imgproc.hpp"
#include "opencv2/objdetect.hpp"
#include <opencv2/highgui/highgui.hpp>

using namespace std;
using namespace cv;
"""

To start here a construction and printing of an opencv Matrix

In [ ]:
icxx"""
const int mySizes[3]={3,5,5};

cv::Mat f = Mat::zeros(3,mySizes,CV_64F);
f.at<double>(1,2,3) = 17;

for ( int i=0;i<mySizes[0];i++) {
    for ( int j=0;j<mySizes[1];j++) {
        for ( int k=0;k<mySizes[2];k++) {
            cerr << f.at<double>(i,j,k) << " ";
        }
        cerr << endl;
    }
    cerr << endl;
}
cerr << endl;
""";

Let's jump to some image processing. Download an example face and load it into opencv.

In [ ]:
; wget -q -O face.png 'https://images.unsplash.com/photo-1554151228-14d9def656e4?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=3633&q=80'

In [ ]:
function cv_grayimage(path)
  cpath = cppstring(path)
  icxx"""
    Mat image = imread($cpath, CV_LOAD_IMAGE_GRAYSCALE);
    image;
  """
end
cv_grayimage("face.png");

In order to inspect whether it worked, we construct a conversion method to get from the opencv matrix to julia.

In [ ]:
using Images: Gray, N0f8

function opencv_to_array2D_superfast(img_opencv)
  if !icxx"$img_opencv.isContinuous();"
     # cloning the opencv image makes it continuous
     img_opencv = icxx"$img_opencv.clone();"
  end

  # we assume 2D
  raw_UInt8 = Base.unsafe_wrap(Array, @cxx(img_opencv->data), (@cxx(img_opencv->cols), @cxx(img_opencv->rows)))
  Gray.(reinterpret(N0f8, raw_UInt8'))  # mind the transpose
end

In [ ]:
opencv_to_array2D_superfast(cv_grayimage("face.png"))

Awesome! We used our first bit of opencv cpp.

On top we now do an histogram equalization for stabilizing light conditions. This is good to do in general, not only for haarcascade face recognition.

In [ ]:
function cv_equalize_inplace(frame)
  icxx"""
    Mat frame = $frame;
    equalizeHist(frame, frame);
    frame;
  """
end
image = cv_equalize_inplace(cv_grayimage("face.png"))
opencv_to_array2D_superfast(image)

The method to load a haarcascade

In [ ]:
function cv_load_cascade(cascade_path)
  icxx"""
    CascadeClassifier cascade;
    cascade.load($(cppstring(cascade_path)));
    return cascade;
  """
end

In [ ]:
; wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml

In [ ]:
cascade_face = cv_load_cascade("$(@__DIR__)/haarcascade_frontalface_alt.xml");

Method to apply a cascade to a image, and collect the resulst as julia rectangles.

In [ ]:
struct Rect
  x::Int
  y::Int
  width::Int
  height::Int
end
Rect(cv_rect) = Rect(
  @cxx(cv_rect->x) + 1,  # julia starts at 1, opencv at 0
  @cxx(cv_rect->y) + 1,  # julia starts at 1, opencv at 0
  @cxx(cv_rect->width),
  @cxx(cv_rect->height))

function cv_detect(cascade, frame_gray, min_scale = 30)
  # faces are always symmetric in recognition
  rects = icxx"""
  std::vector<Rect> rects;
  $cascade.detectMultiScale($frame_gray, rects, 1.1, 2, 0|cv::CASCADE_SCALE_IMAGE, Size($min_scale, $min_scale));
  rects;"""
  [Rect(rect) for rect in rects]
end

In [ ]:
face_rect = cv_detect(cascade_face, image)

Finally again some plotting helpers

In [ ]:
using ImageDraw
function draw_rect(img, rect, color)
  x, y, width, height = rect.x, rect.y, rect.width - 1, rect.height - 1
  points = [Point(x, y), Point(x + width, y), Point(x + width, y + height), Point(x, y + height)]
  draw!(img, Polygon(points), color)
end
function draw_rects(img, rects, color)
  foreach(r -> draw_rect(img, r, color), rects)
  img
end

In [ ]:
img = opencv_to_array2D_superfast(image)
draw_rects(img, face_rect, Gray{N0f8}(1))

# Cpp FFI - it is your turn!

Task 1. Looking at the folder https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/ you will see that opencv comes with a couple of different haarcascades, not only for face recognition.

Download some of them and try them out!

In [ ]:
# TODO

Task 2. Download another image with several faces on it and try the same there

In [ ]:
# TODO

# The End

Thank you very much for participating in this Julia As A Glue Language hands-on!
I hope you learned something and can appreciate like me how impressively easy these three Foreign Function Interfaces are used.

Hope to see you again at the next Julia User Group Munich Meetup!

<center>
<font size="+2">
SAVE THE DATE: <b>Wednesday 10.06.</b>
</font>
</center>

Please, take a minute and leave your feedback on Meetups, how you liked the meetup.

For any suggestions you are always welcome to contact me at s.sahm@reply.de